## Topic Analysis of Review Data

#### DESCRIPTION

Help a leading mobile brand understand the voice of the customer by analyzing the reviews of their product on Amazon and the topics that customers are talking about. You will perform topic modeling on specific parts of speech. You’ll finally interpret the emerging topics.

#### Problem Statement: 

A popular mobile phone brand, Lenovo has launched their budget smartphone in the Indian market. The client wants to understand the VOC (voice of the customer) on the product. This will be useful to not just evaluate the current product, but to also get some direction for developing the product pipeline. The client is particularly interested in the different aspects that customers care about. Product reviews by customers on a leading e-commerce site should provide a good view.

**Domain:** Amazon reviews for a leading phone brand

#### Analysis to be done: 
POS tagging, topic modeling using LDA, and topic interpretation

**Dataset:** ‘K8 Reviews v0.2.csv’

**Sentiment:** The sentiment against the review (4,5 star reviews are positive, 1,2 are negative)

**Reviews:** The main text of the review

#### Steps to perform:

Discover the topics in the reviews and present it to business in a consumable format. Employ techniques in syntactic processing and topic modeling.

Perform specific cleanup, POS tagging, and restricting to relevant POS tags, then, perform topic modeling using LDA. Finally, give business-friendly names to the topics and make a table for business.

#### Tasks: 

1. Read the .csv file using Pandas. Take a look at the top few records.

2. Normalize casings for the review text and extract the text into a list for easier manipulation.

3. Tokenize the reviews using NLTKs word_tokenize function.

4. Perform parts-of-speech tagging on each sentence using the NLTK POS tagger.

5. For the topic model, we should  want to include only nouns.

  - Find out all the POS tags that correspond to nouns.

  - Limit the data to only terms with these tags.

6. Lemmatize. 

  - Different forms of the terms need to be treated as one.

  - No need to provide POS tag to lemmatizer for now.

7. Remove stopwords and punctuation (if there are any). 

8. Create a topic model using LDA on the cleaned up data with 12 topics.

  - Print out the top terms for each topic.

  - What is the coherence of the model with the c_v metric?

9. Analyze the topics through the business lens.

  - Determine which of the topics can be combined.

10. Create topic model using LDA with what you think is the optimal number of topics

  - What is the coherence of the model?

11. The business should  be able to interpret the topics.

  - Name each of the identified topics.

  - Create a table with the topic name and the top 10 terms in each to present to the  business.

In [19]:
import numpy as np
import pandas as pd
import re
import nltk
import gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

#### 1. Read the data

In [2]:
data = pd.read_csv('K8 Reviews v0.2.csv')

In [3]:
data.head()

,sentiment,review
0,1,Good but need updates and improvements
1,0,"Worst mobile i have bought ever, Battery is dr..."
2,1,when I will get my 10% cash back.... its alrea...
3,1,Good
4,0,The worst phone everThey have changed the last...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14675 entries, 0 to 14674
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  14675 non-null  int64 
 1   review     14675 non-null  object
dtypes: int64(1), object(1)
memory usage: 229.4+ KB


#### 2. Normalizing the casing to lower

In [5]:
data['review_cleaned'] = data['review'].apply(lambda review: review.lower())

In [6]:
data.head()

,sentiment,review,review_cleaned
0,1,Good but need updates and improvements,good but need updates and improvements
1,0,"Worst mobile i have bought ever, Battery is dr...","worst mobile i have bought ever, battery is dr..."
2,1,when I will get my 10% cash back.... its alrea...,when i will get my 10% cash back.... its alrea...
3,1,Good,good
4,0,The worst phone everThey have changed the last...,the worst phone everthey have changed the last...


#### 3. Tokenize the reviews using NLTKs word_tokenize function.

In [7]:
reviews = data['review_cleaned']
reviews = [nltk.word_tokenize(review) for review in reviews]

In [8]:
len(reviews)

14675

In [9]:
pprint(reviews[:5], compact=True)

[['good', 'but', 'need', 'updates', 'and', 'improvements'],
 ['worst', 'mobile', 'i', 'have', 'bought', 'ever', ',', 'battery', 'is',
  'draining', 'like', 'hell', ',', 'backup', 'is', 'only', '6', 'to', '7',
  'hours', 'with', 'internet', 'uses', ',', 'even', 'if', 'i', 'put', 'mobile',
  'idle', 'its', 'getting', 'discharged.this', 'is', 'biggest', 'lie', 'from',
  'amazon', '&', 'lenove', 'which', 'is', 'not', 'at', 'all', 'expected', ',',
  'they', 'are', 'making', 'full', 'by', 'saying', 'that', 'battery', 'is',
  '4000mah', '&', 'booster', 'charger', 'is', 'fake', ',', 'it', 'takes', 'at',
  'least', '4', 'to', '5', 'hours', 'to', 'be', 'fully', 'charged.do', "n't",
  'know', 'how', 'lenovo', 'will', 'survive', 'by', 'making', 'full', 'of',
  'us.please', 'don', ';', 't', 'go', 'for', 'this', 'else', 'you', 'will',
  'regret', 'like', 'me', '.'],
 ['when', 'i', 'will', 'get', 'my', '10', '%', 'cash', 'back', '...', '.',
  'its', 'already', '15', 'january..'],
 ['good'],
 ['the', 

#### 4. Perform parts-of-speech tagging on each document using the NLTK POS tagger

In [10]:
%%time
review_pos_tags = [nltk.pos_tag(doc) for doc in reviews]
pprint(review_pos_tags[:5], compact=True)

[[('good', 'JJ'), ('but', 'CC'), ('need', 'VBP'), ('updates', 'NNS'),
  ('and', 'CC'), ('improvements', 'NNS')],
 [('worst', 'JJS'), ('mobile', 'NN'), ('i', 'NN'), ('have', 'VBP'),
  ('bought', 'VBN'), ('ever', 'RB'), (',', ','), ('battery', 'NN'),
  ('is', 'VBZ'), ('draining', 'VBG'), ('like', 'IN'), ('hell', 'NN'),
  (',', ','), ('backup', 'NN'), ('is', 'VBZ'), ('only', 'RB'), ('6', 'CD'),
  ('to', 'TO'), ('7', 'CD'), ('hours', 'NNS'), ('with', 'IN'),
  ('internet', 'JJ'), ('uses', 'NNS'), (',', ','), ('even', 'RB'), ('if', 'IN'),
  ('i', 'JJ'), ('put', 'VBP'), ('mobile', 'JJ'), ('idle', 'NN'),
  ('its', 'PRP$'), ('getting', 'VBG'), ('discharged.this', 'NN'), ('is', 'VBZ'),
  ('biggest', 'JJS'), ('lie', 'NN'), ('from', 'IN'), ('amazon', 'NN'),
  ('&', 'CC'), ('lenove', 'NN'), ('which', 'WDT'), ('is', 'VBZ'), ('not', 'RB'),
  ('at', 'IN'), ('all', 'DT'), ('expected', 'VBN'), (',', ','), ('they', 'PRP'),
  ('are', 'VBP'), ('making', 'VBG'), ('full', 'JJ'), ('by', 'IN'),
  ('saying', 'V

#### 5. Including only nouns for building the Topic Model

In [11]:
review_nouns = [[token for token, pos in doc if pos.startswith('NN')] for doc in review_pos_tags]
pprint(review_nouns[:5], compact=True)

[['updates', 'improvements'],
 ['mobile', 'i', 'battery', 'hell', 'backup', 'hours', 'uses', 'idle',
  'discharged.this', 'lie', 'amazon', 'lenove', 'battery', 'charger', 'hours',
  'don'],
 ['i', '%', 'cash', 'january..'], [],
 ['phone', 'everthey', 'phone', 'problem', 'amazon', 'phone', 'amazon']]


#### 6. Lemmatize

In [12]:
%%time
review_lemmatized = [[nltk.stem.WordNetLemmatizer().lemmatize(token) for token in doc] for doc in review_nouns]
pprint(review_lemmatized[:5])

[['update', 'improvement'],
 ['mobile',
  'i',
  'battery',
  'hell',
  'backup',
  'hour',
  'us',
  'idle',
  'discharged.this',
  'lie',
  'amazon',
  'lenove',
  'battery',
  'charger',
  'hour',
  'don'],
 ['i', '%', 'cash', 'january..'],
 [],
 ['phone', 'everthey', 'phone', 'problem', 'amazon', 'phone', 'amazon']]
Wall time: 1.8 s


#### 7. Remove stopwords and punctuation (if there are any).

In [13]:
%%time
review_no_sw = [[token for token in doc if token not in nltk.corpus.stopwords.words('english') 
                 and token.isalpha()] 
                for doc in review_lemmatized]
pprint(review_no_sw[:5])

[['update', 'improvement'],
 ['mobile',
  'battery',
  'hell',
  'backup',
  'hour',
  'us',
  'idle',
  'lie',
  'amazon',
  'lenove',
  'battery',
  'charger',
  'hour'],
 ['cash'],
 [],
 ['phone', 'everthey', 'phone', 'problem', 'amazon', 'phone', 'amazon']]
Wall time: 16.2 s


#### 8. Create a topic model using LDA on the cleaned up data with 12 topics.

In [15]:
# Build a Dictionary - association word to numeric id
dictionary = gensim.corpora.Dictionary(review_no_sw)
# Transform the collection of texts to a numerical form
corpus = [dictionary.doc2bow(text) for text in review_no_sw]

In [21]:
NUM_TOPICS = 12
lda_model = gensim.models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

In [22]:
print("LDA Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    pprint("Topic #{}: {}".format(idx, lda_model.print_topic(idx, 10)), compact=True)

LDA Model:
('Topic #0: 0.141*"battery" + 0.076*"problem" + 0.066*"phone" + 0.063*"camera" '
 '+ 0.059*"performance" + 0.046*"heating" + 0.041*"backup" + 0.025*"life" + '
 '0.022*"issue" + 0.019*"h"')
('Topic #1: 0.055*"screen" + 0.038*"call" + 0.035*"superb" + 0.032*"app" + '
 '0.027*"camera" + 0.023*"android" + 0.022*"photo" + 0.021*"battery" + '
 '0.020*"stock" + 0.019*"apps"')
('Topic #2: 0.328*"product" + 0.049*"amazon" + 0.030*"delivery" + '
 '0.024*"glass" + 0.023*"return" + 0.018*"service" + 0.018*"replacement" + '
 '0.017*"experience" + 0.016*"condition" + 0.016*"issue"')
('Topic #3: 0.459*"phone" + 0.038*"problem" + 0.023*"handset" + 0.019*"box" + '
 '0.019*"piece" + 0.012*"gallery" + 0.012*"smartphone" + 0.011*"r" + '
 '0.010*"cable" + 0.009*"feature"')
('Topic #4: 0.051*"time" + 0.050*"phone" + 0.025*"work" + 0.022*"video" + '
 '0.020*"earphone" + 0.020*"super" + 0.019*"data" + 0.018*"call" + '
 '0.018*"headphone" + 0.017*"day"')
('Topic #5: 0.300*"mobile" + 0.052*"hai" + 0.

In [23]:
pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.132280  0.026601       1        1  14.185736
0      0.113082 -0.096640       2        1  12.034122
6     -0.009639  0.118459       3        1  11.760621
9      0.025956 -0.057711       4        1   8.920110
11     0.104951 -0.040078       5        1   8.632637
8      0.034425  0.121846       6        1   7.471906
2     -0.203257  0.094671       7        1   7.348303
3      0.051232  0.189988       8        1   6.867156
4      0.027725  0.017959       9        1   5.922180
1      0.081340 -0.113558      10        1   5.871169
7     -0.052842 -0.204135      11        1   5.595562
5     -0.305252 -0.057402      12        1   5.390504, topic_info=        Term         Freq        Total Category  logprob  loglift
47   product  2158.000000  2158.000000  Default  30.0000  30.0000
15     phone  6938.000000  6938.000000  Default  29.0000  29.0000
11    mobile  1635.000000  1635.000000  Default  28.0000  28.0000
55    camera  3257.000000  3257.000000  Default  27.0000  27.0000
4    battery  3076.000000  3076.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
15     phone    67.750221  6938.750000  Topic12  -4.1572  -1.7085
92   feature    42.824276   861.412354  Topic12  -4.6159  -0.0809
490  company    30.571445   134.745834  Topic12  -4.9530   1.4372
782     heat    32.815998   382.667023  Topic12  -4.8821   0.4643
23    lenovo    34.496861   890.963257  Topic12  -4.8322  -0.3309

[747 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1367      3  0.061019        aap
1367      7  0.915283        aap
1328      1  0.965505     access
332       8  0.692672  accessory
332       9  0.293054  accessory
...     ...       ...        ...
1024      4  0.030253  yesterday
1024      8  0.937850  yesterday
1464      2  0.923400         yr
1210      9  0.914666       zone
1770      5  0.953074       zoom

[1945 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 1, 7, 10, 12, 9, 3, 4, 5, 2, 8, 6])

In [24]:
# Compute Coherence Score
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=review_no_sw, dictionary=dictionary, coherence='c_v')

coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.5423933187777923


#### 8.1 Print out the top terms for each topic.

#### 8.2 coherence of the model with the c_v metric

#### 9. Analyze the topics through the business lens - Determine which of the topics can be combined

In [ ]:
custom_stop_words = ['phone', 'problem', 'issue', 'product', 'mobile']

#### 10. Create topic model using LDA with what you think is the optimal number of topics

#### 10.1 Coherence of the optimal model

#### 11. Business interpretation of the identified topics

#### 11.1. Table with the topic names and the top 10 terms in each to present to the business